In [12]:

import json
import pandas as pd
from pathlib import Path


In [13]:
# Defining paths
group_data = Path("/Users/harshul/website clone/harshul/test/group-data/")

MEMBERS_DIR_PATH = group_data / 'members/'
WEBSITE_DATA_PATH = group_data / 'website_data/'
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / 'content'


In [14]:
pd.DataFrame(experiences +education)
# if MSU in insti and no end_date

,institution,city,state,country,role,start_date,end_date,degree,subject
0,Ohio state University,,Ohio,USA,Research assistant (astronomy),2016-01-01,2019-12-31,NaN,NaN
1,Michigan State University,East Lansing,Michigan,USA,NaN,2019-01-01,NaN,PhD,Physics and Astronomy.
2,Vassar College,Poughkeepsie,New York,USA,NaN,2012-01-01,2016-12-31,Bachelors,"Physics, Astronomy, and Russian Studies"


In [19]:
def extract_member_data(member_dir):
    if member_dir.name.startswith('.'):
        return None

    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'


    if (member_experiences := (member_json_dir / 'experiences.json')).exists():
        experiences = json.load(open(member_experiences))
    else:
        experiences = []


    if (member_education := (member_json_dir / 'education.json')).exists():
        education = json.load(open(member_education))
    else:
        education = []


    return member_record, experiences, education


def create_education_experience_df(experiences, education):
    ex_edu_df = pd.DataFrame(experiences + education)
    return ex_edu_df

def process_education_experience(ex_edu_df):
    # Drop rows where 'institution' is NaN
    ex_edu_df = ex_edu_df.dropna(subset=['institution'])
    
    filtered_df = ex_edu_df[ex_edu_df['institution'].str.contains('Michigan State University')]
    return filtered_df

def assign_academic_role(df):
    if df.empty:
        return 'Other'

    top_record = df.iloc[0]

    if  top_record.get('role', '') == 'Assistant Professor' or top_record.get('role', '')=='Professor':
        return 'Professor'
    if top_record.get('role', '') in ['Visiting Researcher', 'Postdoctoral Researcher']:
        return 'Postdoctoral researcher'
    if top_record.get('degree', '') == 'Bachelors':
        return 'Graduate Student' if top_record.get('end_date') else 'Undergraduate Student'
    if top_record.get('degree', '') in ['Masters', 'PhD']:
        return 'Graduate Student'
    else:
        return 'Other'

member_records = []

for member_dir in MEMBERS_DIR_PATH.glob('*'):
    if member_dir.name.startswith('.'):
        continue

    member_record, experiences, education = extract_member_data(member_dir)

    education_experience_df = create_education_experience_df(experiences, education)
    
    academic_role = assign_academic_role(education_experience_df)

    nick_name = member_record.get('nick_name', '')
    first_name = member_record.get('first_name', '')
    last_name = member_record.get('last_name', '')

    name = f"{nick_name or first_name} {last_name}"


    member_records.append({"Name": name, "Academic Role": academic_role})

people_page_df = pd.DataFrame(member_records)
people_page_df

,Name,Academic Role
0,Josh Shields,Other
1,Anirban Dutta,Postdoctoral researcher
2,Erin Visser,Undergraduate Student
3,Vicente Amado Olivo,Graduate Student
4,Yuki Matsumura,Graduate Student
5,Andrew Fullard,Other
6,Isaac Smith,Graduate Student
7,Hayden Monk,Undergraduate Student
8,Atharva Arya,Graduate Student
9,Tripp Dow,Other


In [16]:
# For people page ####################################################################
def extract_member_data(member_dir):
    if member_dir.name.startswith('.'):
        return None

    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'
    
    top_experience_role = ""
    top_education_degree = ""
    top_education_end_date = None
  
    if (member_experiences := (member_json_dir / 'experiences.json')).exists():
        experiences = json.load(open(member_experiences))
        top_experience_role = experiences[0].get('role', "")
    
    if (member_education := (member_json_dir / 'education.json')).exists():
        education = json.load(open(member_education))

        top_education_degree = education[0].get('degree', "")
        top_education_end_date = education[0].get('end_date')
    

    return member_record, top_experience_role, top_education_degree, top_education_end_date


def assign_academic_role(top_experience_role, top_education_degree, top_education_end_date):
    if 'Assistant Professor' in top_experience_role or 'Professor' in top_experience_role:
        return 'Professor'
    elif top_experience_role in ['Visiting Researcher', 'Postdoctoral Researcher']:
        return 'Postdoctoral researcher'
    elif top_education_degree == 'Bachelors':
        return 'Graduate Student' if top_education_end_date is not None else 'Undergraduate Student'
    elif top_education_degree in ['Masters', 'PhD']:
        return 'Graduate Student'
    else:
        return 'Other'

member_records = []

for member_dir in MEMBERS_DIR_PATH.glob('*'):
    if member_dir.name.startswith('.'):
        continue

    member_record, top_experience_role, top_education_degree, top_education_end_date = extract_member_data(member_dir)
    
    academic_role = assign_academic_role(top_experience_role, top_education_degree, top_education_end_date)

    nick_name = member_record.get('nick_name', None)
    second_name = member_record.get('last_name', "")
    if not nick_name:
        first_name = member_record.get('first_name', "")
        last_name = member_record.get('last_name', "")
        name = f"{first_name} {last_name}"
    else:
        name = f"{nick_name} {second_name}"

    member_records.append({"Name": name, "Academic Role": academic_role})

people_page_df = pd.DataFrame(member_records)
people_page_df


,Name,Academic Role
0,Josh Shields,Graduate Student
1,Anirban Dutta,Postdoctoral researcher
2,Erin Visser,Undergraduate Student
3,Vicente Amado Olivo,Graduate Student
4,Yuki Matsumura,Graduate Student
5,Andrew Fullard,Graduate Student
6,Isaac Smith,Graduate Student
7,Hayden Monk,Undergraduate Student
8,Atharva Arya,Graduate Student
9,Tripp Dow,Undergraduate Student
